In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

In [ ]:
# Загрузка модели
model_name = "ez3nx/ru-tpulse-finance-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
df = pd.read_excel('/content/df_to_annotate_2.xlsx')
df.shape

(370387, 7)

In [ ]:
df.head(2)

,post_id,inserted,processed_posts,reactions_count,comments_count,total_day_posts_cnt,ticker
0,2d8173d7-7a65-4401-951f-20a055ab8a06,2025-02-28,В автоследование вложил минималку возможную. О...,0,0,301,SBER
1,9f4d42b6-2319-4663-ab88-5169d42de15e,2025-02-28,"{$SBER}: Разбираем отчёт Сбербанка - плюсы, ми...",1,0,301,SBER


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = model.to(device)

cuda


In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm
import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def predict_sentiment_3class_with_ids(
    df, text_column, id_column, model, tokenizer, batch_size=128, device="cuda"
):
    results = []
    model.eval()
    problematic_records = []

    # Создаем список кортежей (id, text) для обработки
    id_text_pairs = list(zip(df[id_column].tolist(), df[text_column].tolist()))

    # Обработка по батчам
    for i in tqdm(range(0, len(id_text_pairs), batch_size)):
        batch_pairs = id_text_pairs[i : i + batch_size]
        batch_ids = [pair[0] for pair in batch_pairs]
        batch_texts = []

        # Проверяем каждый текст в батче перед токенизацией
        for idx, (post_id, text) in enumerate(
            zip(batch_ids, [pair[1] for pair in batch_pairs])
        ):
            try:
                # Обработка None или не-строковых значений
                if text is None:
                    logger.warning(
                        f"Обнаружен None в тексте для post_id={post_id}. Заменено на пустую строку."
                    )
                    batch_texts.append("")
                elif not isinstance(text, str):
                    converted = str(text)
                    logger.warning(
                        f"Post_id={post_id}: Текст не является строкой (тип: {type(text)}). Преобразовано в строку: '{converted}'"
                    )
                    batch_texts.append(converted)
                else:
                    batch_texts.append(text)
            except Exception as e:
                error_msg = f"Ошибка при обработке post_id={post_id}: {str(e)}"
                logger.error(error_msg)
                problematic_records.append(
                    {"post_id": post_id, "error": str(e), "text": str(text)}
                )
                batch_texts.append(
                    ""
                )  # Добавляем пустую строку вместо проблемного текста

        try:
            # Токенизация
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=512,
            )

            # Перемещаем входные данные на GPU
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Предсказание
            with torch.no_grad():
                outputs = model(**inputs)

            probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
            pred_classes = torch.argmax(probabilities, dim=-1).cpu().tolist()

            # Маппинг на текстовые метки
            sentiment_map = {
                0: "negative",
                1: "neutral",
                2: "positive",
            }

            # Сохраняем результаты вместе с исходными ID
            for j, (label_idx, post_id, text) in enumerate(
                zip(pred_classes, batch_ids, batch_texts)
            ):
                results.append(
                    {
                        "post_id": post_id,
                        "text": text,
                        "sentiment": sentiment_map[label_idx],
                        "label": label_idx,
                    }
                )
        except Exception as e:
            error_msg = f"Ошибка при обработке батча {i // batch_size + 1}: {str(e)}"
            logger.error(error_msg)
            # Перебираем все элементы в проблемном батче и добавляем их в список проблемных записей
            for post_id, text in zip(batch_ids, batch_texts):
                problematic_records.append(
                    {"post_id": post_id, "error": str(e), "text": text}
                )

    # Сохраняем информацию о проблемных записях
    if problematic_records:
        problem_df = pd.DataFrame(problematic_records)
        logger.info(f"Всего проблемных записей: {len(problematic_records)}")
        problem_df.to_csv("problematic_records.csv", index=False)
        logger.info("Проблемные записи сохранены в файл 'problematic_records.csv'")

    # Создаем DataFrame с результатами
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
sentiment_results = predict_sentiment_3class_with_ids(
    df,
    text_column='processed_posts',
    id_column='post_id',
    model=model,
    tokenizer=tokenizer
)

  0%|          | 0/2894 [00:00<?, ?it/s]

In [ ]:
sentiment_results.to_excel("labeled_data_final.xlsx", index=False) # encoding='utf-8',

In [ ]:
sentiment_results

,post_id,text,sentiment,label
0,2d8173d7-7a65-4401-951f-20a055ab8a06,В автоследование вложил минималку возможную. О...,negative,0
1,9f4d42b6-2319-4663-ab88-5169d42de15e,"{$SBER}: Разбираем отчёт Сбербанка - плюсы, ми...",positive,2
2,a23081b3-6568-441c-b679-7a67de67ee71,{$SBER} .🤣🤣,neutral,1
3,6050d49d-c47d-4fde-899b-1c9ade242e18,{$NVTK} {$NVTK} {$GAZP} {$SBER} {$USDRUBF}\nВа...,positive,2
4,d6e330aa-0a04-4080-b322-d80285c3fb3d,{$SBER}\n{$SPH5}\n{$SRM5}\nФьючерсы на обыкнов...,neutral,1
5,d6c2e992-02f7-4936-be50-11449ddd209e,{$NVTK} {$GAZP} {$SBER} {$USDRUBF} \nЗначит ...,neutral,1
6,ce65ee37-a193-4d3e-bfa2-b75ff9b0681f,{$SBER} эх сольют уж понятно... Европа полюбэ ...,negative,0
7,f6165335-9b62-4cda-a073-ecf324f6f729,{$SBER} а что будет если подпишут документ? Э...,neutral,1
8,a8b0b5cb-5bd4-4f16-9cbe-93c857b0e358,Анализ Сбера {$SBER} {$SBERP}\n\nБумаги снизил...,positive,2
9,e48486e2-4f1a-4b93-a536-49440cc179ca,"{$SBER} Донни, спасибо! Это лучшее, что можно ...",positive,2


In [ ]:
sentiment_results.text.values

array(['{$SBER} мб Сургут сбер решил выкупить))))))',
       '•{$MTLR} Мечел (+0,9%). На последней волне вверх широкого рынка данные бумаги шли вниз, но как только бенчмарк пошел в откат они вдруг стали расти. Накануне отмечался там именно вариант вверх без предупреждения. Для такой высоковолатильной акции вчерашние 1% ни о чем: они перепроданы, и вариант высокого отскока не снимается с повестки; технически скорее вверх вплоть до 120 руб., чем вниз и в очередной раз к 100 руб.',
       '🛢️Так же мне до сих пор не доплатили живы по {$LKOH} Лукойл, по моим меркам примерно 61.52 руб. На обращение в поддержку написали - ждите до 24, ну а 25 выплатили почти 46 руб. дивов и сразу же списали почти 53 руб. Цирк одним словом, снова написал в поддержку и сижу жду ответа уже второй день!',
       '{$SGZH} пздц такое движение не увидел,там прям по графику было видно. \nВы реально странные кто шорты берет в пружине. Да конечно на этом такие как я будут зарабатывать деньги. Но все равно надо видеть.